In [29]:
import cv2
import numpy as np
import copy

# cv2.namedWindow('mask')

video_path = "03_BackgroundSubstraction/unicycle.mp4"
cam = cv2.VideoCapture(video_path)

_, background = cam.read()  #Забираем первый кадр с фоном
background = cv2.cvtColor(background, cv2.COLOR_BGR2HSV) #Перевод в HSV

while (True):
    success, frame = cam.read()
    
    if(success == False):
        cam.release()
        cam = cv2.VideoCapture(video_path)  
        continue #Проверка взятия кадра из видео
    
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    background = cv2.addWeighted(background, 0.97, hsv_frame, 0.03, 0) #На выходе - еле заметное очертание человека
    
    diff = cv2.absdiff(hsv_frame,background) #Остается только очертание человека
    foreground_mask = cv2.inRange(diff, (0, 18, 20), (255, 240, 200)) #Маска разницы на основе анализа каналов HSV
    
    kernel = np.ones((13, 13), np.uint8)
    morph = cv2.morphologyEx(foreground_mask, cv2.MORPH_CLOSE, kernel) #Морфология
    
    connectivity = 4 
    output = cv2.connectedComponentsWithStats(morph, connectivity, cv2.CV_32S)

    num_labels = output[0]
    labels = output[1]
    stats = output[2]
    
    filtered = np.zeros_like(morph)
    
    for i in range(1, num_labels):
        a = stats[i, cv2.CC_STAT_AREA]
        top = stats[i, cv2.CC_STAT_TOP]
        left = stats[i, cv2.CC_STAT_LEFT]
        width = stats[i, cv2.CC_STAT_WIDTH]
        height = stats[i, cv2.CC_STAT_HEIGHT]
        
        if (a >= 7000):
            filtered[np.where(labels == i)] = 255
            print(a) 
            
            cv2.putText(frame, str(a / (width * height))[:4], (left, top), cv2.FONT_ITALIC, 1, (0, 0, 255), 2, cv2.LINE_4)
            cv2.rectangle(frame, (left, top), (left + width, top + height), (0, 255, 0), 2) #Анализ компонент связности
    
    # cv2.imshow('frame', hsv_frame[:, :, 2])
    # cv2.imshow('back', background[:, :, 2])
    
    # cv2.imshow('diff0', diff[:, :, 0])
    # cv2.imshow('diff1', diff[:, :, 1])
    # cv2.imshow('diff2', diff[:, :, 2])
    
    cv2.imshow('for_mask', foreground_mask)
    cv2.imshow('morph', morph)
    cv2.imshow('mask_filtered', filtered)
    
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(60) & 0xFF
    
    if (key == ord('q')):
        break

cam.release()
cv2.destroyAllWindows()
cv2.waitKey(10)

11368
15187
17761
20614
22812
25752
22282
20667
21542
22911
23455
22841
21547
20872
9036
9574
17767
17069
19215
22904
25324
26236
22893
25089
25885
26216
25824
24759
23466
25882
25904
24640
11873
13669
25056
24833
25636
23256
22694
9592
12505
25642
26894
27152
25695
26484
27407
25657
24833
24895
25205
26247
27342
26679
27754
29279
29912
28800
25338
23269
25451
28184
27175
26036
25836
24312
26039
22815
23772
9696
10698
7628
10372
14198
16912
19479
21405
25467
22384
21952
18607
24696
23534
23484
21599
21563
11830
20253
19656
21046
21133
26321
25040
21564
24413
24343
24370
23923
24250
21892
24876
25565
24140
12526
12294
23440
23684
25319
24022
23477
10910
12614
25576
27930
27786
26694
25841
25037
25136
24026
24156
24691
26819
28294
28249
31429
33781
29182
29899
29092
28144
26949
29181
25662
24810
15644
7973
23573
24696
21885
22991
9197
11778
8564
10258
7959
16817
19404
21018
25947
23625
23357
20698
26294
25271
24359
22623
13412
8759
12594
20277
19216
21925
22955
26123
24007
22188
24714
23

-1